In [133]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt


## Data Preparation



In [134]:
from olist.data import Olist

data = Olist().get_data()
data.keys()


dict_keys(['sellers', 'product_category_name_translation', 'orders', 'order_items', 'customers', 'geolocation', 'order_payments', 'order_reviews', 'products'])

In [135]:
from olist.seller import Seller

sellers1 = Seller().get_training_data()
sellers = sellers1.copy()
sellers.head()


,seller_id,seller_city,seller_state,delay_to_carrier,wait_time,date_first_sale,date_last_sale,months_on_olist,n_orders,quantity,quantity_per_order,sales,review_score,share_of_five_stars,share_of_one_stars
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,0.000000,13.018588,2017-05-05 16:25:11,2017-08-30 12:50:19,4.0,3,3,1.000,218.70,3.000000,0.333333,0.333333
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0.000000,9.065716,2017-03-29 02:10:34,2018-06-06 20:15:21,14.0,40,41,1.025,11703.07,4.560976,0.731707,0.048780
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ,0.000000,4.042292,2018-07-30 12:44:49,2018-07-30 12:44:49,0.0,1,1,1.000,158.00,5.000000,1.000000,NaN
3,c0f3eea2e14555b6faeea3dd58c1b1c3,sao paulo,SP,0.000000,5.667187,2018-08-03 00:44:08,2018-08-03 00:44:08,0.0,1,1,1.000,79.99,5.000000,1.000000,NaN
4,51a04a8a6bdcb23deccc82b0b80742cf,braganca paulista,SP,3.353727,35.314861,2017-11-14 12:15:25,2017-11-14 12:15:25,0.0,1,1,1.000,167.99,1.000000,NaN,1.000000


In [141]:
sellers_df = (
            Seller().get_seller_features()
            .merge(Seller().get_seller_delay_wait_time(), on="seller_id")
            .merge(Seller().get_active_dates(), on="seller_id")
            .merge(Seller().get_quantity(), on="seller_id")
            .merge(Seller().get_sales(), on="seller_id")
        )
sellers_df


,seller_id,seller_city,seller_state,delay_to_carrier,wait_time,date_first_sale,date_last_sale,months_on_olist,n_orders,quantity,quantity_per_order,sales
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,0.000000,13.018588,2017-05-05 16:25:11,2017-08-30 12:50:19,4.0,3,3,1.000000,218.70
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0.000000,9.065716,2017-03-29 02:10:34,2018-06-06 20:15:21,14.0,40,41,1.025000,11703.07
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ,0.000000,4.042292,2018-07-30 12:44:49,2018-07-30 12:44:49,0.0,1,1,1.000000,158.00
3,c0f3eea2e14555b6faeea3dd58c1b1c3,sao paulo,SP,0.000000,5.667187,2018-08-03 00:44:08,2018-08-03 00:44:08,0.0,1,1,1.000000,79.99
4,51a04a8a6bdcb23deccc82b0b80742cf,braganca paulista,SP,3.353727,35.314861,2017-11-14 12:15:25,2017-11-14 12:15:25,0.0,1,1,1.000000,167.99
...,...,...,...,...,...,...,...,...,...,...,...,...
2965,98dddbc4601dd4443ca174359b237166,sarandi,PR,0.000000,8.230822,2018-07-14 03:10:19,2018-07-23 15:34:03,0.0,2,2,1.000000,158.00
2966,f8201cab383e484733266d1906e2fdfa,palhoca,SC,0.000000,13.045797,2017-06-17 15:30:11,2018-08-11 19:55:17,14.0,10,10,1.000000,889.00
2967,74871d19219c7d518d0090283e03c137,sao paulo,SP,0.000000,3.606625,2018-04-05 17:09:46,2018-08-21 19:25:15,5.0,6,7,1.166667,550.04
2968,e603cf3fec55f8697c9059638d6c8eb5,pelotas,RS,0.000000,6.657634,2017-02-09 02:43:17,2017-06-16 17:30:16,4.0,8,11,1.375000,297.00


In [142]:
freight_sellers = (
    data["order_items"][["seller_id", "freight_value"]]
    .groupby("seller_id")
    .sum()
    .rename(columns={"freight_value": "sales_freight"})
)
#print(freight_sellers.head(3))
sellers_df = sellers_df.merge(freight_sellers, on="seller_id")
sellers_df.head(3)


,seller_id,seller_city,seller_state,delay_to_carrier,wait_time,date_first_sale,date_last_sale,months_on_olist,n_orders,quantity,quantity_per_order,sales,sales_freight
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,0.0,13.018588,2017-05-05 16:25:11,2017-08-30 12:50:19,4.0,3,3,1.000,218.70,27.90
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0.0,9.065716,2017-03-29 02:10:34,2018-06-06 20:15:21,14.0,40,41,1.025,11703.07,1438.73
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ,0.0,4.042292,2018-07-30 12:44:49,2018-07-30 12:44:49,0.0,1,1,1.000,158.00,16.21


Edit the `get_training_data` method so that the DataFrame it returns contains the fields:
- `revenues`: sum of subscription and sales fees revenues
- `cost_of_reviews`: sum of costs associated with bad reviews
- `profits`: `revenues` - `cost_of_reviews`

In [143]:
## Revenues

sellers_df["sales_fees"] = 0.01 * (sellers_df["sales"] - sellers_df["sales_freight"])

condition = sellers_df["months_on_olist"] > 0
sellers_df.loc[condition, "subscription_fees"] = 80 * sellers_df["months_on_olist"]
sellers_df.loc[~condition, "subscription_fees"] = 80

sellers_df["revenues"] = sellers_df["sales_fees"] + sellers_df["subscription_fees"]
sellers_df.head(5)


,seller_id,seller_city,seller_state,delay_to_carrier,wait_time,date_first_sale,date_last_sale,months_on_olist,n_orders,quantity,quantity_per_order,sales,sales_freight,sales_fees,subscription_fees,revenues
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,0.000000,13.018588,2017-05-05 16:25:11,2017-08-30 12:50:19,4.0,3,3,1.000,218.70,27.90,1.9080,320.0,321.9080
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0.000000,9.065716,2017-03-29 02:10:34,2018-06-06 20:15:21,14.0,40,41,1.025,11703.07,1438.73,102.6434,1120.0,1222.6434
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ,0.000000,4.042292,2018-07-30 12:44:49,2018-07-30 12:44:49,0.0,1,1,1.000,158.00,16.21,1.4179,80.0,81.4179
3,c0f3eea2e14555b6faeea3dd58c1b1c3,sao paulo,SP,0.000000,5.667187,2018-08-03 00:44:08,2018-08-03 00:44:08,0.0,1,1,1.000,79.99,15.66,0.6433,80.0,80.6433
4,51a04a8a6bdcb23deccc82b0b80742cf,braganca paulista,SP,3.353727,35.314861,2017-11-14 12:15:25,2017-11-14 12:15:25,0.0,1,1,1.000,167.99,31.93,1.3606,80.0,81.3606


In [144]:
reviews_df = data["order_reviews"].copy()
reviews_df.head(3)


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24


In [130]:
#sellers2 = sellers.copy()
sellers.drop(columns=["review_score"], inplace=True)
sellers.head(3)


,seller_id,seller_city,seller_state,delay_to_carrier,wait_time,date_first_sale,date_last_sale,months_on_olist,n_orders,quantity,quantity_per_order,sales,share_of_five_stars,share_of_one_stars,sales_freight,sales_fees,subscription_fees,revenues
0,3442f8959a84dea7ee197c632cb2df15,campinas,SP,0.0,13.018588,2017-05-05 16:25:11,2017-08-30 12:50:19,4.0,3,3,1.000,218.70,0.333333,0.333333,27.90,1.9080,320.0,321.9080
1,d1b65fc7debc3361ea86b5f14c68d2e2,mogi guacu,SP,0.0,9.065716,2017-03-29 02:10:34,2018-06-06 20:15:21,14.0,40,41,1.025,11703.07,0.731707,0.048780,1438.73,102.6434,1120.0,1222.6434
2,ce3ad9de960102d0677a81f5d0bb7b2d,rio de janeiro,RJ,0.0,4.042292,2018-07-30 12:44:49,2018-07-30 12:44:49,0.0,1,1,1.000,158.00,1.000000,NaN,16.21,1.4179,80.0,81.4179


In [168]:
## Cost of reviews: sum of costs associated with bad reviews(review_score <= 3 )
orders = data["orders"].copy()
order_items = data["order_items"].copy()

# Merge sellers1 and order_items on seller_id.
merged_df = pd.merge(sellers_df, order_items, on="seller_id")
# Then, merge the result with orders on order_id.
merged_df = pd.merge(merged_df, orders, on="order_id")
# Finally, merge the result with review_score_df on order_id.
merged_df = pd.merge(merged_df, reviews_df, on="order_id")

merged_df = merged_df[
    [
        "seller_id",
        "sales",
        "sales_freight",
        "sales_fees",
        "subscription_fees",
        "revenues",
        "review_score",
        "n_orders",
    ]
]

merged_df.head()


,seller_id,sales,sales_freight,sales_fees,subscription_fees,revenues,review_score,n_orders
0,3442f8959a84dea7ee197c632cb2df15,218.70,27.90,1.9080,320.0,321.9080,5,3
1,3442f8959a84dea7ee197c632cb2df15,218.70,27.90,1.9080,320.0,321.9080,1,3
2,3442f8959a84dea7ee197c632cb2df15,218.70,27.90,1.9080,320.0,321.9080,3,3
3,8c16d1f32a54d92897cc437244442e1b,5642.60,2112.07,35.3053,1840.0,1875.3053,3,108
4,d1b65fc7debc3361ea86b5f14c68d2e2,11703.07,1438.73,102.6434,1120.0,1222.6434,5,40



Monetary cost for each bad review:
```python
# review_score: cost(BRL)
{'1 star': 100
'2 stars': 50
'3 stars': 40
'4 stars': 0
'5 stars': 0}
```

In [169]:
merged_df.loc[merged_df['review_score'] == 1, 'review_costs'] = 100
merged_df.loc[merged_df['review_score'] == 2, 'review_costs'] = 50
merged_df.loc[merged_df['review_score'] == 3, 'review_costs'] = 40
merged_df.loc[merged_df['review_score'] == 4, 'review_costs'] = 0
merged_df.loc[merged_df['review_score'] == 5, 'review_costs'] = 0
#merged_df1 = merged_df.drop_duplicates(subset=['seller_id'], keep='first')
merged_df.head(10)


,seller_id,sales,sales_freight,sales_fees,subscription_fees,revenues,review_score,n_orders,review_costs
0,3442f8959a84dea7ee197c632cb2df15,218.70,27.90,1.9080,320.0,321.9080,5,3,0.0
1,3442f8959a84dea7ee197c632cb2df15,218.70,27.90,1.9080,320.0,321.9080,1,3,100.0
2,3442f8959a84dea7ee197c632cb2df15,218.70,27.90,1.9080,320.0,321.9080,3,3,40.0
3,8c16d1f32a54d92897cc437244442e1b,5642.60,2112.07,35.3053,1840.0,1875.3053,3,108,40.0
4,d1b65fc7debc3361ea86b5f14c68d2e2,11703.07,1438.73,102.6434,1120.0,1222.6434,5,40,0.0
5,d1b65fc7debc3361ea86b5f14c68d2e2,11703.07,1438.73,102.6434,1120.0,1222.6434,5,40,0.0
6,d1b65fc7debc3361ea86b5f14c68d2e2,11703.07,1438.73,102.6434,1120.0,1222.6434,5,40,0.0
7,d1b65fc7debc3361ea86b5f14c68d2e2,11703.07,1438.73,102.6434,1120.0,1222.6434,5,40,0.0
8,d1b65fc7debc3361ea86b5f14c68d2e2,11703.07,1438.73,102.6434,1120.0,1222.6434,5,40,0.0
9,d1b65fc7debc3361ea86b5f14c68d2e2,11703.07,1438.73,102.6434,1120.0,1222.6434,5,40,0.0


__Sum up the review costs per seller__

In [170]:
abc = merged_df.groupby('seller_id')['review_costs'].sum().reset_index()
merged_df1 = pd.merge(merged_df, abc, on="seller_id")
merged_df1 = merged_df1.drop_duplicates(subset=['seller_id'], keep='first').drop(columns=['review_costs_x'])

merged_df1 =  merged_df1.rename(columns={'review_costs_y': 'review_costs'})
merged_df1.head(10)


,seller_id,sales,sales_freight,sales_fees,subscription_fees,revenues,review_score,n_orders,review_costs
0,3442f8959a84dea7ee197c632cb2df15,218.70,27.90,1.9080,320.0,321.9080,5,3,140.0
3,8c16d1f32a54d92897cc437244442e1b,5642.60,2112.07,35.3053,1840.0,1875.3053,3,108,1330.0
119,d1b65fc7debc3361ea86b5f14c68d2e2,11703.07,1438.73,102.6434,1120.0,1222.6434,5,40,240.0
160,ce3ad9de960102d0677a81f5d0bb7b2d,158.00,16.21,1.4179,80.0,81.4179,5,1,0.0
161,c0f3eea2e14555b6faeea3dd58c1b1c3,79.99,15.66,0.6433,80.0,80.6433,5,1,0.0
162,51a04a8a6bdcb23deccc82b0b80742cf,167.99,31.93,1.3606,80.0,81.3606,1,1,100.0
163,c240c4061717ac1806ae6ee72be3533b,59.90,13.99,0.4591,80.0,80.4591,4,1,0.0
164,a5a1bfcf728ab0e19182959cf0771ee4,1812.96,347.89,14.6507,560.0,574.6507,4,21,260.0
188,9906e7aa42a3c067b7c9a564d01c4320,2619.00,496.60,21.2240,480.0,501.2240,4,12,200.0
204,e49c26c3edfa46d227d5121a6b6e4d37,3654.25,1175.46,24.7879,1440.0,1464.7879,3,35,370.0


__Sort sellers by increasing profits(revenues - costs),starting with sellers with the lowest profits.__

In [178]:
merged_df1['profits'] = merged_df1['revenues'] - merged_df1['review_costs']
merged_df1 = merged_df1.sort_values(by='profits', ascending=True)
merged_df1


,seller_id,sales,sales_freight,sales_fees,subscription_fees,revenues,review_score,n_orders,review_costs,profits
73053,7c67e1448b00f6e969d365cea6b010ab,187923.89,51612.55,1363.1134,1440.0,2803.1134,2,982,44290.0,-41486.8866
21816,4a3ca9315b744ce9f8e9374361493884,200472.92,35067.04,1654.0588,1600.0,3254.0588,2,1806,44620.0,-41365.9412
1392,6560211a19b47992c3666cc44a7e94c0,123304.83,27960.94,953.4389,1440.0,2393.4389,1,1854,40120.0,-37726.5611
30625,1f50f920176fa81dab994f9023523100,106939.21,35165.77,717.7344,1360.0,2077.7344,1,1404,37490.0,-35412.2656
35612,1025f0e2d44d7041d6cf58b6550e0bfa,138968.55,33892.14,1050.7641,1120.0,2170.7641,1,915,31290.0,-29119.2359
...,...,...,...,...,...,...,...,...,...,...
111744,9df5750209b83567ec144eb299d7d634,4917.00,259.30,46.5770,1680.0,1726.5770,4,11,200.0,1526.5770
111235,612a743d294c27884fb7b80d2b19ba35,3057.70,104.45,29.5325,1520.0,1549.5325,4,2,0.0,1549.5325
3435,f7496d659ca9fdaf323c0aae84176632,5087.57,844.94,42.4263,1760.0,1802.4263,5,26,190.0,1612.4263
96731,989becdce12ebc39863c2bceab6f3ca1,7840.88,1052.45,67.8843,1760.0,1827.8843,5,18,140.0,1687.8843


__Method to update the IT costs after removing sellers along with the products that they sell.__

In [164]:
import math
def get_IT_score(data_for_IT_costs):
    n_sellers = merged_df1['seller_id'].nunique()
    print('Total number of unique sellers in the dataset: ', n_sellers)

    n_products = merged_df1['n_orders'].sum()
    print('Total number of products ever sold by sellers in the dataset: ', n_products)

    alpha = 3157.27
    beta = 978.23

    IT_costs = alpha * math.sqrt(n_sellers) + beta * math.sqrt(n_products)
    print('Total IT costs: ', IT_costs)
    print("Total profits: ", merged_df1['profits'].sum())
    return IT_costs


__Remove sellers with low profits alongwith their products__

In [179]:
import math

# implement a loop that removes the sellers starting with the least profitable
# seller from 'sellers_sorted' until the total IT costs are equal 500k BRL.
it_costs = 0
iteration = 0
#it_costs <= 500000
for i, row in merged_df1.iterrows():
    if row.profits < 0:
        print('Iteration: ', iteration)
        merged_df1 = merged_df1.drop(i)
        it_costs = get_IT_score(merged_df1)
        iteration += 1
        print("*"*50)
    else:
        break


Iteration:  0
Total number of unique sellers in the dataset:  2966
Total number of products ever sold by sellers in the dataset:  98859
Total IT costs:  479521.68629105855
Total profits:  -342070.8350000002
**************************************************
Iteration:  1
Total number of unique sellers in the dataset:  2965
Total number of products ever sold by sellers in the dataset:  97053
Total IT costs:  476670.30215448077
Total profits:  -300704.89379999985
**************************************************
Iteration:  2
Total number of unique sellers in the dataset:  2964
Total number of products ever sold by sellers in the dataset:  95199
Total IT costs:  473716.4466682293
Total profits:  -262978.3327
**************************************************
Iteration:  3
Total number of unique sellers in the dataset:  2963
Total number of products ever sold by sellers in the dataset:  93795
Total IT costs:  471453.50498451455
Total profits:  -227566.0671
*******************************

" while merged_df1['profits'] < 0 and merged_df1['seller_id'].nunique() > 0:\n    print('Iteration: ', iteration)\n    merged_df1 = merged_df1.iloc[1:]\n    it_costs = get_IT_score(merged_df1)\n    iteration += 1 "

In [180]:
merged_df1


,seller_id,sales,sales_freight,sales_fees,subscription_fees,revenues,review_score,n_orders,review_costs,profits
32865,5fd924b4836098a5be0ecf81ba054ce0,1148.80,135.54,10.1326,80.0,90.1326,5,7,90.0,0.1326
92788,7c462184585ae80bd6e5774c62ce50f2,60.00,17.64,0.4236,80.0,80.4236,3,1,80.0,0.4236
87452,b8d890108a7a80e4f5d59eea9f362438,89.70,34.59,0.5511,80.0,80.5511,4,2,80.0,0.5511
89358,ef728fa1f17436c91ed1ccd03dcf9631,128.70,58.12,0.7058,80.0,80.7058,5,3,80.0,0.7058
67458,31561f325664a8a7aba4c8d0c3a9b3db,2702.19,627.17,20.7502,160.0,180.7502,3,36,180.0,0.7502
...,...,...,...,...,...,...,...,...,...,...
111744,9df5750209b83567ec144eb299d7d634,4917.00,259.30,46.5770,1680.0,1726.5770,4,11,200.0,1526.5770
111235,612a743d294c27884fb7b80d2b19ba35,3057.70,104.45,29.5325,1520.0,1549.5325,4,2,0.0,1549.5325
3435,f7496d659ca9fdaf323c0aae84176632,5087.57,844.94,42.4263,1760.0,1802.4263,5,26,190.0,1612.4263
96731,989becdce12ebc39863c2bceab6f3ca1,7840.88,1052.45,67.8843,1760.0,1827.8843,5,18,140.0,1687.8843


In [181]:
merged_df1.shape


(2222, 10)